In [91]:
import pandas as pd
import numpy as np
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense

In [92]:
def binomial (n, p, size):
    """
    Generate random numbers following a binomial distribution.

    Parameters:
    - n:    Number of trials (number of times an event is repeated).
    - p:    Probability of success for each trial.
    - size: Number of times to repeat the binomial experiment.

    Returns:
    - An array of random numbers representing the number of successes in each experiment.
    """
    return np.random.binomial(n, p, size)

def randint (low, high, size):
    """
    Generate random integers from a discrete uniform distribution.

    Parameters:
    - low:  The minimum (inclusive) value of the range.
    - high: The maximum (exclusive) value of the range.
    - size: The number of random integers to generate.

    Returns:
    - An array of random integers within the specified range.
    """
    return np.random.randint (low, high, size)

# QUESTION 1
There are n products to be produced $\newline$
There are m suppliers which provide the parts to produce the products $\newline$
A product i needs $a_{ij}$ parts ($a_{ij >= 0}$, $i = 1\rightarrow n$, $j=1\rightarrow m $) $\newline$
The preorder cost for part $j$ is $b_j$ $\newline$ 
The demand for the products is random vector $D = (D_1,..., D_n)$ $\newline$
Cost additionaly $l_i$ to satisfy a unit of demand for product $i$, and unit selling price of that unit is $q_i$ $\newline$
The number of parts ordered is $x_j$, the number of units produced is $z_i$, the numbers of remaining parts unused is $y_j$


### Second stage
The second stage is mentioned first to emphasize the importance of understanding the uncertainty in case of solving a stochastic problem. This approach aligns with the principles of stochastic programming, where the second-stage decisions are made after observing the realization of random data and are contingent on the first-stage decisions. $\newline$
In this stage, we want to minimize the cost of production. Specifically, we need to find the optimal cost of filling an additional portion of demand while reselling the salvage parts to minimize the loss of not using the parts. $\newline$
$$ 
min_{x,y} \text { } Z=\sum^n_{i=1} (l_i-q_i)z_i - \sum^m_{j=1}s_jy_j \newline \text {subject to} \newline
y_j=x_j- \sum_{i=1}^n a_{ij}z_i, j=1,...,m \newline
0 \leq z_i \leq d_i, i=1,...,n \newline
y_j \geq 0, j=1,...,m
$$

### First stage
In this stage, $x$ needs to be decided before a realization of the demand $D$, which makes it a $\textit {here and now}$ decisions $\newline$
Moreover, the expected value of $Z$ should also be taken into account. To be specific, that is the optimal value $Q(x)$ of the second stage ($Q(x) = E[Z(z,y)]$) $\newline$
Therefore, we yield this problem below: 
$$min \text { } g(x,y,z) = b^Tx + Q(x) = b^Tx + E[Z(z,y)]$$

### Big problem
We need to solve this
$$
min \text { } b^Tx + \sum^S_{i=1} p_s[(l_i-q_i)^Tz_i - s^T_iy_i] \newline
\text {subject to} \newline
y \geq 0 \newline
0 \leq z \leq d \newline
x \geq 0 \newline
$$

In [93]:
n   = 8           # number of products
m   = 5           # number of parts to be ordered
S   = 2           # number of scenarios
p_s = [0.5, 0.5]  # probability of each scenario / density

D = np.array([binomial (10, 0.5, n),
              binomial (10, 0.5, n)]) # demand for each product in each scenario matrix [n x S]

b = randint(5, 10, m) # preorder cost for each part

A = []
for i in range(n):
    A.append(np.array(randint(1,10,m)))

A = np.array(A) # bill of materials matrix [m x n]
# A = np.transpose(A)

s = randint (1, 10, m)       # salvage values
l = randint (100, 200, n)    # additional costs
q = randint (1000, 1400, n)  # unit selling prices

c = l - q #cost coefficients

In [94]:
print('Preorder cost: ', b)
print('Material usage:\n', A)
print('Salvage value: ', s)
print('Additional producing cost: ', l)
print('Selling prices: ', q)
print('Cost coefficient:', c)
# print('Demand:\n', D)

Preorder cost:  [7 6 7 9 9]
Material usage:
 [[4 1 1 8 2]
 [5 8 3 9 4]
 [6 3 6 1 8]
 [7 5 3 3 1]
 [1 5 9 8 6]
 [1 2 7 3 1]
 [1 6 9 2 9]
 [6 1 4 8 2]]
Salvage value:  [1 2 2 2 5]
Additional producing cost:  [104 145 152 193 148 122 167 177]
Selling prices:  [1382 1278 1239 1087 1179 1084 1266 1383]
Cost coefficient: [-1278 -1133 -1087  -894 -1031  -962 -1099 -1206]


In [95]:
# df_parts_usage = []
# for i in range(m):
#     df_parts_usage.append([PARTS_NAME[i]])
#     for j in range(n):
#         df_parts_usage[i].append(A[i][j])

# df_parts_usage = pd.DataFrame(
#     data = df_parts_usage,
#     columns=[''] + PRODUCTS_NAME
# ).set_index('')

# df_parts_usage

In [96]:
m = Container()


In [97]:
products = ['product1', 'product2', 'product3', 'product4', 'product5','product6','product7','product8']
# products = [0,1,2,3,4,5,6,7]
i = Set(
    container=m,
    name='products',
    description='products of the firm',
    records=products
)
i.records

uni element_text
0  product1             
1  product2             
2  product3             
3  product4             
4  product5             
5  product6             
6  product7             
7  product8

In [98]:
parts = ['part1', 'part2', 'part3', 'part4', 'part5']
# parts = [0,1,2,3,4]
j = Set(
    container=m,
    name='parts',
    description='Parts to build the products',
    records=parts
)
j.records

uni element_text
0  part1             
1  part2             
2  part3             
3  part4             
4  part5

In [99]:
B = Parameter(
    container=m,
    name='Preorder_price',
    domain=j,
    description='Price to preorder a part',
    records=b
)
B.records

parts  value
0  part1    7.0
1  part2    6.0
2  part3    7.0
3  part4    9.0
4  part5    9.0

In [100]:
a = Parameter (
    container=m,
    name='parts_usages',
    domain=[i, j],
    description='Numbers of parts needed for n products',
    records=A
)
a.records

products  parts  value
0   product1  part1    4.0
1   product1  part2    1.0
2   product1  part3    1.0
3   product1  part4    8.0
4   product1  part5    2.0
5   product2  part1    5.0
6   product2  part2    8.0
7   product2  part3    3.0
8   product2  part4    9.0
9   product2  part5    4.0
10  product3  part1    6.0
11  product3  part2    3.0
12  product3  part3    6.0
13  product3  part4    1.0
14  product3  part5    8.0
15  product4  part1    7.0
16  product4  part2    5.0
17  product4  part3    3.0
18  product4  part4    3.0
19  product4  part5    1.0
20  product5  part1    1.0
21  product5  part2    5.0
22  product5  part3    9.0
23  product5  part4    8.0
24  product5  part5    6.0
25  product6  part1    1.0
26  product6  part2    2.0
27  product6  part3    7.0
28  product6  part4    3.0
29  product6  part5    1.0
30  product7  part1    1.0
31  product7  part2    6.0
32  product7  part3    9.0
33  product7  part4    2.0
34  product7  part5    9.0
35  product8  part1    6.0
36  product8  part2    1.0
37  product8  part3    4.0
38  product8  part4    8.0
39  product8  part5    2.0

In [101]:
sc = Set(
    container=m,
    name="scenario",
    description=" adf ad",
    records=["s1", "s2"]
)
sc.records

uni element_text
0  s1             
1  s2

In [102]:
p = Parameter(
    container=m,
    name="scenario_p",
    domain=sc
)
p[sc] = 0.5
p.records

scenario  value
0       s1    0.5
1       s2    0.5

In [103]:
d = Parameter(
    container=m,
    name='Demand',
    domain=[sc,i],
    description="Demand for products",
    records = D
)

d.records

scenario  products  value
0        s1  product1    5.0
1        s1  product2    4.0
2        s1  product3    7.0
3        s1  product4    4.0
4        s1  product5    5.0
5        s1  product6    5.0
6        s1  product7    3.0
7        s1  product8    6.0
8        s2  product1    4.0
9        s2  product2    5.0
10       s2  product3    3.0
11       s2  product4    6.0
12       s2  product5    5.0
13       s2  product6    4.0
14       s2  product7    3.0
15       s2  product8    5.0

In [104]:
sal = Parameter(
    container=m,
    name='Salvage_price',
    domain=j,
    records=s
)
sal.records

parts  value
0  part1    1.0
1  part2    2.0
2  part3    2.0
3  part4    2.0
4  part5    5.0

In [105]:
Apc = Parameter(
    container=m,
    name='Addition_producing_cost',
    domain=i,
    records = l
)
Apc.records

products  value
0  product1  104.0
1  product2  145.0
2  product3  152.0
3  product4  193.0
4  product5  148.0
5  product6  122.0
6  product7  167.0
7  product8  177.0

In [106]:
Sp = Parameter(
    container=m,
    name='Selling_price',
    domain=i,
    records=q
)
Sp.records

products   value
0  product1  1382.0
1  product2  1278.0
2  product3  1239.0
3  product4  1087.0
4  product5  1179.0
5  product6  1084.0
6  product7  1266.0
7  product8  1383.0

In [107]:
x = Variable(
    container=m,
    name='x',
    domain=j,
    type="positive",
    description='Number of preordered parts'
)

z = Variable(
    container=m,
    name='z',
    domain=i,
    type="positive",
    description='Number of produced products'
)

y = Variable(
    container=m,
    name='y',
    domain=j,
    type="positive",
    description='Number of unused parts'
)

In [108]:
# con_1_1 = Equation(
#     container=m,
#     name='Constraint_1_1',
#     domain=j,
#     definition= x>=0
# )

# con_1_2 = Equation(
#     container=m,
#     name='Constraint_1_2',
#     domain=j,
#     definition=y>=0
# )

# con_1_3 = Equation(
#     container=m,
#     name='Constraint_1_3',
#     domain=i,
#     definition=z>=0
# )

con_1_4 = Equation(
    container=m,
    name='Constraint_1_4',
    domain=j
)
con_1_4[j] = x[j] - Sum((i,j), a[i,j]*z[i])

con_1_5 = Equation(
    container=m,
    name='Constraint_1_5',
    domain=i,
    definition= z <= d
)

$$
min \text { } b^Tx + \sum^S_{s=1} p_s[\sum_{i=1}^n(l_i-q_i)z_i - \sum_{j=1}^ms_jy_j]$$

In [109]:
obj = Sum(j, B[j]*x[j]) + Sum(sc, p[sc]* (Sum(i, (Apc[i]-Sp[i])*z[i]) - Sum(j, sal[j]*y[j])))

In [110]:
production = Model(
    container=m,
    name='production',
    equations=m.getEquations(),
    problem='LP',
    sense=Sense.MIN,
    objective=obj
)

In [111]:
import sys
production.solve(output=sys.stdout)

--- Job _job_df5b97fc-8429-443e-959c-78964379a441.gms Start 12/10/23 14:50:54 45.4.0 19dc3313 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\DUC_AN\.conda\envs\gamspy\Lib\site-packages\gamspy_base\gmsprmNT.txt
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP CONOPT
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS CONOPT
    DNLP CONOPT
    RMINLP CONOPT
    MINLP SBB
    QCP CONOPT
    MIQCP SBB
    RMIQCP CONOPT
    EMP CONVERT
    Restart C:\Users\DUC_AN\AppData\Local\Temp\tmptn4t9ted\_save_381d38fe-5899-4585-b6fb-dd487e17607c.g00
    Input C:\Users\DUC_AN\AppData\Local\Temp\tmptn4t9ted\_job_df5b97fc-8429-443e-959c-78964379a441.gms
    Output C:\Users\DUC_AN\AppData\Local\Temp\tmptn4t9ted\_job_df5b97fc-8429-443e-959c-78964379a441.lst
    Save C:\Users\DUC_AN\AppData\Local\Temp\tmptn4t9ted\_restart_381d38fe-5899-4585-b6fb-dd487e17607c.g00
    ScrDir C:\Users\DUC_AN\AppData\Local\Temp\tmptn4t9ted\225a\
    SysDir C:\Users\DUC_AN\.conda\envs\ga

GamsExceptionExecution: 

================================================================================
Error Summary
================================================================================
  39  Constraint_1_4(parts) .. (x(parts) - sum((products,parts),(parts_usages(products,parts) * z(products)))) =e= 0;
****                                                          $125
**** LINE     20 INPUT       C:\Users\DUC_AN\AppData\Local\Temp\tmptn4t9ted\_job_df5b97fc-8429-443e-959c-78964379a441.gms
**** 125  Set is under control already

GAMS return code not 0 (2), check C:\Users\DUC_AN\AppData\Local\Temp\tmptn4t9ted\_job_df5b97fc-8429-443e-959c-78964379a441.lst for more details